This journal investigates the effects of a simulation with and without frames using gendaylit1axis

In [1]:
import bifacial_radiance
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
bifacial_radiance.__version__

'0.3.4+124.g805ac4e.dirty'

#### Control variables

In [3]:
smallsim = True
shamsul = False


In [4]:
testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'FrameTest')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\FrameTest


In [5]:
if shamsul:
    TMYtoread = r'C:\Users\sarefeen\Documents\RadianceScenes\Temp\SRRL_WeatherFile_TMY3_60_2020.csv'
else:
    TMYtoread = r'C:\Users\sayala\Documents\GitHub\RTCanalysis\TEMP\SRRL_WeatherFile_TMY3_60_2020_FIXED.csv'

In [6]:
#### Reading the weatherfile
'''
weatherfile = pd.read_csv(TMYtoread, header = 1)
weatherfile.head()
wf2 = weatherfile[weatherfile['DNI (W/m^2)'] == weatherfile['DNI (W/m^2)'].max()]
wf3 = weatherfile[weatherfile['Date (MM/DD/YYYY)']== '4/29/2020']
y = wf3['DNI (W/m^2)']
x = wf3['Time (HH:MM)']
plt.plot(x,y)
plt.xticks(rotation = 45)
weatherfile.groupby('Date (MM/DD/YYYY)')['DNI (W/m^2)'].sum().max()
''';

### Make Modules

In [7]:
simulationname = 'OmegaTestField'
moduletype_framed='Framed_Panel'
moduletype_simple='NotFramed_Panel'

numpanels = 1  
x = 1  
y = 2
lat=39.742 # NREL SSRL location
lon=-105.179 # NREL SSRL location
elev=1829
timezone=-7
axis_tilt=0
axis_azimuth=180
limit_angle=60
backtrack=True 
gcr=0.35
angledelta=0.01
numpanels=1
torquetube=True
diameter = 0.130175        # 5 1/8 in
torqueTubeMaterial='Metal_Grey'
tubetype='Round'

# for torquetube, the simulation 
axisofrotationTorqueTube = False  # This is particular to the NREL site.
hub_height = 1.35    
xgap = 0.01    # 1 cm
zgap = 0.05    # 1 inch of arm, + 1 3/16 of panel width on average ~ 0.055 m
pitch=5.7      # distance between rows

sensorsy = 3 # Increase sampling for edge 

In [8]:
# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = True
cellLevelModule = False

In [9]:
albedo = 0.2  #'grass'     # ground albedo


#this change is for smalling the simulation
if smallsim:
    nMods = 1 
    nRows = 1 
    sensorsy = 9
else:
    nMods = 20 
    nRows = 10 
    sensorsy = 9
cumulativesky = False

In [10]:
demo = bifacial_radiance.RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object'

path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\FrameTest


In [11]:
frameParams = {'frame_material' : 'Metal_Grey', 
               'frame_thickness' : 0.003,
               'frame_z' : 0.03,
               'nSides_frame' : 4,
               'frame_width' : 0.05}


omegaParams = {'omega_material': 'litesoil',
                'x_omega1' : 0.08,
                'mod_overlap' : 0.03,
                'y_omega' : 0.75,
                'x_omega3' : 0.02,
                'omega_thickness' : 0.01,
                'inverted' : False}
moduleDict=demo.makeModule(name=moduletype_simple,x=x,y=y,numpanels = numpanels, xgap=xgap, zgap=zgap,
                            torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=torqueTubeMaterial,
                            axisofrotationTorqueTube=axisofrotationTorqueTube, omegaParams = None, frameParams = None)

moduleDict=demo.makeModule(name=moduletype_framed,x=x,y=y,numpanels = numpanels, xgap=xgap, zgap=zgap,
                            torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=torqueTubeMaterial,
                            axisofrotationTorqueTube=axisofrotationTorqueTube, omegaParams = omegaParams, frameParams = frameParams)


Module Name: NotFramed_Panel
Pre-existing .rad file objects\NotFramed_Panel.rad will be overwritten
Postion 1 Offsetfromaxis: 0
Module NotFramed_Panel updated in module.json

Module Name: Framed_Panel
Pre-existing .rad file objects\Framed_Panel.rad will be overwritten
it is getting in to make omega
Postion 1 Offsetfromaxis: 0
Module Framed_Panel updated in module.json


## A. SIMPLE Module run

In [12]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
startdate = '20_01_01_12'      
enddate = '20_01_01_12'
trackerdict = demo.gendaylit1axis(startdate = startdate, enddate = enddate) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_simple, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
result = demo.analysis1axis(customname='test_A')

print("\n TRACKER TILT:", demo.trackerdict['20_01_01_12_00']['surf_tilt']) # Sanity checkt of surface tilt



start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/

Making 1 octfiles in root directory.
Created 1axis_20_01_01_12_00.oct
Linescan in process: 1axis_20_01_01_12_00test_A_Front
Linescan in process: 1axis_20_01_01_12_00test_A_Back
Saved: results\irr_1axis_20_01_01_12_00test_A.csv
Index: 20_01_01_12_00. Wm2Front: 115.58932222222221. Wm2Back: 18.52196037037037
Saving a cumulative-results file in the main simulation folder.This adds up by sensor location the irradiance over all hours or configurations considered.
Not able to save a cumulative result for th

## B. With Frame Test

In [13]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
startdate = '20_01_01_12'      
enddate = '20_01_01_12'
trackerdict = demo.gendaylit1axis(startdate = startdate, enddate = enddate) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_framed, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
print(demo.trackerdict['20_01_01_12_00']['surf_tilt'])
result = demo.analysis1axis(customname='test_B', sensorsy=sensorsy)


start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/

Making 1 octfiles in root directory.
Created 1axis_20_01_01_12_00.oct
16.9
Linescan in process: 1axis_20_01_01_12_00test_B_Front
Linescan in process: 1axis_20_01_01_12_00test_B_Back
Saved: results\irr_1axis_20_01_01_12_00test_B.csv
Index: 20_01_01_12_00. Wm2Front: 115.50528888888887. Wm2Back: 18.265116296296295
Saving a cumulative-results file in the main simulation folder.This adds up by sensor location the irradiance over all hours or configurations considered.
Not able to save a cumulative result 

## Compare Results

In [14]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00test.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,4.724416e-17,1.137531,1.116481,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5430,19.315693,0.167172
1,0.580193,3.552657e-17,1.195672,1.174622,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5510,19.216177,0.166299
2,0.388830,2.380898e-17,1.253812,1.232762,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5592,19.116547,0.165425
3,0.197467,1.209139e-17,1.311953,1.290903,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5674,17.555773,0.151908
4,0.006105,3.738079e-19,1.370093,1.349043,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5756,12.519877,0.108325
5,-0.185258,-1.134378e-17,1.428234,1.407184,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5838,19.134087,0.165542
6,-0.376621,-2.306137e-17,1.486374,1.465324,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5920,20.061153,0.173550
7,-0.567983,-3.477895e-17,1.544514,1.523465,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6002,20.252513,0.175193
8,-0.759346,-4.649654e-17,1.602655,1.581605,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5541,20.574037,0.178045


In [15]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00test_B.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,4.724416e-17,1.137531,1.116481,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4364,18.702120,0.162011
1,0.580193,3.552657e-17,1.195672,1.174622,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4489,18.290343,0.158427
2,0.388830,2.380898e-17,1.253812,1.232762,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4620,18.184940,0.157496
3,0.197467,1.209139e-17,1.311953,1.290903,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4750,16.910577,0.146442
4,0.006105,3.738079e-19,1.370093,1.349043,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4881,13.646790,0.118165
5,-0.185258,-1.134378e-17,1.428234,1.407184,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5011,18.880390,0.163464
6,-0.376621,-2.306137e-17,1.486374,1.465324,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5142,19.558607,0.169316
7,-0.567983,-3.477895e-17,1.544514,1.523465,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.6059,19.919567,0.172304
8,-0.759346,-4.649654e-17,1.602655,1.581605,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.6160,20.292713,0.175517


# C. Modify Scanning position, Not Framed

In [17]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
singleindex = '20_01_01_12'      
singleindex = '20_01_01_12'
trackerdict = demo.gendaylit1axis(startdate = singleindex, enddate = singleindex) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_simple, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
print(demo.trackerdict['20_01_01_12_00']['surf_tilt'])

scanpoints = int(0.05/0.005)     # sample 5 cm from the edge, with a resolution of 0.005 mm 
frame_thickness = 0.01
modscanBack = {}
for ii in range(1, 3):
    modscanBack['ystart']  = x/2.0 - (frame_thickness + 0.001) - 0.005*ii # (adding frame thicknes plus 1 mm so it does not overlay exactly) 
    result = demo.analysis1axis(singleindex=singleindex+'_00', modscanfront=modscanBack, modscanback=modscanBack, relative = False, customname='_test_C_pos_'+str(ii), debug=True)
#    result = demo.analysis1axis(modscanfront=modscanBack, modscanback=modscanBack, relative = False, customname='_test_C_pos_'+str(ii))

    

start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/

Making 1 octfiles in root directory.
Created 1axis_20_01_01_12_00.oct
16.9
For debug:
 hub_height, Azimuth, Tilt, nMods, nRows, Pitch, Offset, SceneY, SceneX
1.35 90.0 16.9 1 1 5.7 0 2.0 1.01
Sampling: modWanted 1, rowWanted 1 out of {nMods} modules, {nRows} rows
Azimuth 90.0
Coordinate Center Point of Desired Panel before azm rotation 0.0 0.0
Coordinate Center Point of Desired Panel after azm rotation 0.0 0.0
Edge of Panel 0.9568135840576074 5.85879346548428e-17 -0.29070219359825245
Offset Shift 0.0

In [18]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_C_pos_1.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,0.484,1.137531,1.116481,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6299,19.549637,0.169069
1,0.580193,0.484,1.195672,1.174622,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6376,19.573627,0.169266
2,0.388830,0.484,1.253812,1.232762,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6453,19.065183,0.164858
3,0.197467,0.484,1.311953,1.290903,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6530,18.709320,0.161770
4,0.006105,0.484,1.370093,1.349043,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6607,20.275197,0.175297
5,-0.185258,0.484,1.428234,1.407184,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6684,20.324713,0.175714
6,-0.376621,0.484,1.486374,1.465324,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6761,20.122207,0.173952
7,-0.567983,0.484,1.544514,1.523465,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4975,20.496473,0.177461
8,-0.759346,0.484,1.602655,1.581605,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5022,20.767727,0.179802


In [19]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_C_pos_2.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,0.479,1.137531,1.116481,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4138,19.655120,0.170300
1,0.580193,0.479,1.195672,1.174622,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4333,19.643860,0.170174
2,0.388830,0.479,1.253812,1.232762,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4529,19.188987,0.166205
3,0.197467,0.479,1.311953,1.290903,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4724,18.691397,0.161868
4,0.006105,0.479,1.370093,1.349043,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4920,19.934360,0.172602
5,-0.185258,0.479,1.428234,1.407184,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5115,20.126537,0.174237
6,-0.376621,0.479,1.486374,1.465324,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5311,20.389157,0.176480
7,-0.567983,0.479,1.544514,1.523465,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5475,20.700593,0.179151
8,-0.759346,0.479,1.602655,1.581605,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5571,20.722823,0.179328


## D. Modifying Scanning Position with Frames

In [20]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
singleindex = '20_01_01_12'      
trackerdict = demo.gendaylit1axis(startdate = singleindex, enddate = singleindex) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_framed, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
print(demo.trackerdict['20_01_01_12_00']['surf_tilt'])

scanpoints = int(0.05/0.005)     # sample 5 cm from the edge, with a resolution of 0.005 mm 
frame_thickness = 0.01
modscanBack = {}
for ii in range(0, 2):
    modscanBack['ystart']  = x/2.0 - (frame_thickness + 0.001) - 0.005*ii # (adding frame thicknes plus 1 mm so it does not overlay exactly) 
    result = demo.analysis1axis(modscanfront=modscanBack, modscanback=modscanBack, relative = False, customname='_test_D_pos_'+str(ii))

    

start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/

Making 1 octfiles in root directory.
Created 1axis_20_01_01_12_00.oct
16.9
Linescan in process: 1axis_20_01_01_12_00_test_D_pos_0_Front
Linescan in process: 1axis_20_01_01_12_00_test_D_pos_0_Back
Saved: results\irr_1axis_20_01_01_12_00_test_D_pos_0.csv
Index: 20_01_01_12_00. Wm2Front: 115.563. Wm2Back: 2.6578834074074074
Saving a cumulative-results file in the main simulation folder.This adds up by sensor location the irradiance over all hours or configurations considered.
Not able to save a cumulati

In [21]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_D_pos_0.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,0.489,1.137531,1.116481,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5143,2.489247,0.021549
1,0.580193,0.489,1.195672,1.174622,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5280,2.341696,0.020269
2,0.388830,0.489,1.253812,1.232762,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5416,2.476027,0.021430
3,0.197467,0.489,1.311953,1.290903,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5553,2.755083,0.023842
4,0.006105,0.489,1.370093,1.349043,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5690,2.554014,0.022099
5,-0.185258,0.489,1.428234,1.407184,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5826,2.818634,0.024386
6,-0.376621,0.489,1.486374,1.465324,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5963,2.815070,0.024352
7,-0.567983,0.489,1.544514,1.523465,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5850,3.016593,0.026098
8,-0.759346,0.489,1.602655,1.581605,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5949,2.654588,0.022964


In [22]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_C_pos_1.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,0.484,1.137531,1.116481,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6299,19.549637,0.169069
1,0.580193,0.484,1.195672,1.174622,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6376,19.573627,0.169266
2,0.388830,0.484,1.253812,1.232762,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6453,19.065183,0.164858
3,0.197467,0.484,1.311953,1.290903,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6530,18.709320,0.161770
4,0.006105,0.484,1.370093,1.349043,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6607,20.275197,0.175297
5,-0.185258,0.484,1.428234,1.407184,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6684,20.324713,0.175714
6,-0.376621,0.484,1.486374,1.465324,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6761,20.122207,0.173952
7,-0.567983,0.484,1.544514,1.523465,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4975,20.496473,0.177461
8,-0.759346,0.484,1.602655,1.581605,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5022,20.767727,0.179802
